# Modelling Appendix

To get the individual breakdown of all models' performance on each role, we run the code with y as a single column rather than as a whole in 02-modelling. This allows us to store all the results for each role as an individual dataframe.

### Contents

- [Traditional Keeper](#Traditional-Keeper)
- [Sweeper Keeper](#Sweeper-Keeper)
- [Ball-Playing Defender](#Ball-Playing-Defender)
- [No-Nonsense Defender](#No-Nonsense-Defender)
- [Full-Back](#Full-Back)
- [All-Action Midfielder](#All-Action-Midfielder)
- [Midfield Playmaker](#Midfield-Playmaker)
- [Traditional Winger](#Traditional-Winger)
- [Inverted Winger](#Inverted-Winger)
- [Goal Poacher](#Goal-Poacher)
- [Target Man](#Target-Man)

## Traditional Keeper

In [14]:
import pandas as pd
from pycaret.classification import setup, compare_models, predict_model, finalize_model, pull
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import joblib

# Load the CSV file into a DataFrame
df = pd.read_csv('../data/players_df_sin_reco.csv')

# Select features (columns 7 to 20, 0-based index: 6 to 19)
X = df.iloc[:, 6:20]

# Select the 11th column from the end as the target
y = df.iloc[:, -11]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an empty DataFrame to store results
traditionalkeeper_df = pd.DataFrame(columns=['model', 'train_accuracy', 'test_accuracy', 'train_f1', 'test_f1', 'TT'])

# Concatenate X_train and y_train into a DataFrame for PyCaret setup
df_target = pd.concat([X_train, y_train], axis=1)

# Setup PyCaret environment
clf = setup(data=df_target, target=y.name, session_id=42)

# Compare different models and store the results
best_model = compare_models()  # This returns the top model
compare_results = pull()  # Pulls all model results as a DataFrame

# Define the mapping from model names to PyCaret identifiers
model_mapping = {
    'K Neighbors Classifier': 'knn',
    'Random Forest Classifier': 'rf',
    'Logistic Regression': 'lr',
    'Decision Tree Classifier': 'dt',
    'Ada Boost Classifier': 'ada',
    'Gradient Boosting Classifier': 'gbc',
    'Naive Bayes': 'nb',
    'SVM - Linear Kernel': 'svm',
    'Extra Trees Classifier': 'et',
    'Light Gradient Boosting Machine': 'lightgbm',
    'Ridge Classifier': 'ridge',
    'Linear Discriminant Analysis': 'lda',
    'Extreme Gradient Boosting': 'xgboost',
    'Dummy Classifier': 'dummy'
}

# Loop over each model in the compare_results DataFrame
for idx, row in compare_results.iterrows():
    model_name = row['Model']
    testing_time = row['TT (Sec)']

    # Get the PyCaret identifier for the model
    model_identifier = model_mapping.get(model_name)

    # Instead of finalizing by the model name, select the model by identifier
    if model_identifier:
        best_model = compare_models(include=[model_identifier], n_select=1)

        # Finalize the model (train it on the entire dataset)
        final_model = finalize_model(best_model)

        # Predict on train and test sets
        predictions_train = predict_model(final_model, data=X_train)
        predictions_test = predict_model(final_model, data=X_test)

        # Calculate test accuracy and F1 score
        y_pred_test = predictions_test['prediction_label']
        test_accuracy = accuracy_score(y_test, y_pred_test)
        test_f1 = f1_score(y_test, y_pred_test, average='weighted')

        # Calculate train accuracy and F1 score
        y_pred_train = predictions_train['prediction_label']
        train_accuracy = accuracy_score(y_train, y_pred_train)
        train_f1 = f1_score(y_train, y_pred_train, average='weighted')

        # Append the metrics for each model to the results DataFrame
        new_row = pd.DataFrame({
            'model': model_name,
            'train_accuracy': train_accuracy,
            'test_accuracy': test_accuracy,
            'train_f1': train_f1,
            'test_f1': test_f1,
            'TT': testing_time
        }, index=[0])

        traditionalkeeper_df = pd.concat([traditionalkeeper_df, new_row], ignore_index=True)


,Description,Value
0,Session id,42
1,Target,Class_Traditional Keeper
2,Target type,Binary
3,Original data shape,"(91, 15)"
4,Transformed data shape,"(91, 15)"
5,Transformed train set shape,"(63, 15)"
6,Transformed test set shape,"(28, 15)"
7,Numeric features,14
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.9833,0.9000,0.9000,0.8500,0.8667,nan,0.8632,0.0220
dt,Decision Tree Classifier,0.9833,0.8900,0.9000,0.8500,0.8667,nan,0.8632,0.0140
ridge,Ridge Classifier,0.9833,0.8800,0.9000,0.8500,0.8667,nan,0.8632,0.0160
rf,Random Forest Classifier,0.9833,0.9000,0.8000,0.8000,0.8000,nan,0.8000,0.0950
lda,Linear Discriminant Analysis,0.9833,0.8800,0.9000,0.8500,0.8667,nan,0.8632,0.0150
ada,Ada Boost Classifier,0.9667,0.8900,0.8000,0.7500,0.7667,nan,0.7632,0.0460
gbc,Gradient Boosting Classifier,0.9667,0.8900,0.8000,0.7500,0.7667,nan,0.7632,0.0460
et,Extra Trees Classifier,0.9667,0.9000,0.8000,0.7500,0.7667,nan,0.7632,0.0720
xgboost,Extreme Gradient Boosting,0.9667,0.9000,0.8000,0.7500,0.7667,nan,0.7632,0.1510
lr,Logistic Regression,0.9524,0.8800,0.8000,0.7000,0.7333,nan,0.7278,1.6280


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.9833,0.9000,0.9000,0.8500,0.8667,nan,0.8632,0.0250


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9524,0.8433,0.6000,0.6000,0.6000,nan,0.6000,0.0380


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.8571,0.4500,0.0000,0.0000,0.0000,nan,0.0000,0.0160


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

In [13]:
traditionalkeeper_df

,model,train_accuracy,test_accuracy,train_f1,test_f1,TT
0,K Neighbors Classifier,0.978022,1.000000,0.978664,1.000000,0.034
1,Decision Tree Classifier,1.000000,1.000000,1.000000,1.000000,0.016
2,Ridge Classifier,0.978022,1.000000,0.978664,1.000000,0.015
3,Random Forest Classifier,1.000000,1.000000,1.000000,1.000000,0.096
4,Linear Discriminant Analysis,0.978022,1.000000,0.978664,1.000000,0.015
5,Ada Boost Classifier,1.000000,1.000000,1.000000,1.000000,0.058
6,Gradient Boosting Classifier,1.000000,1.000000,1.000000,1.000000,0.080
7,Extra Trees Classifier,1.000000,1.000000,1.000000,1.000000,0.083
8,Extreme Gradient Boosting,1.000000,1.000000,1.000000,1.000000,0.025
9,Logistic Regression,1.000000,1.000000,1.000000,1.000000,0.030


## Sweeper Keeper

In [15]:
import pandas as pd
from pycaret.classification import setup, compare_models, predict_model, finalize_model, pull
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import joblib

# Load the CSV file into a DataFrame
df = pd.read_csv('../data/players_df_sin_reco.csv')

# Select features (columns 7 to 20, 0-based index: 6 to 19)
X = df.iloc[:, 6:20]

# Select the 10th column from the end as the target
y = df.iloc[:, -10]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an empty DataFrame to store results
sweeperkeeper_df = pd.DataFrame(columns=['model', 'train_accuracy', 'test_accuracy', 'train_f1', 'test_f1', 'TT'])

# Concatenate X_train and y_train into a DataFrame for PyCaret setup
df_target = pd.concat([X_train, y_train], axis=1)

# Setup PyCaret environment
clf = setup(data=df_target, target=y.name, session_id=42)

# Compare different models and store the results
best_model = compare_models()  # This returns the top model
compare_results = pull()  # Pulls all model results as a DataFrame

# Define the mapping from model names to PyCaret identifiers
model_mapping = {
    'K Neighbors Classifier': 'knn',
    'Random Forest Classifier': 'rf',
    'Logistic Regression': 'lr',
    'Decision Tree Classifier': 'dt',
    'Ada Boost Classifier': 'ada',
    'Gradient Boosting Classifier': 'gbc',
    'Naive Bayes': 'nb',
    'SVM - Linear Kernel': 'svm',
    'Extra Trees Classifier': 'et',
    'Light Gradient Boosting Machine': 'lightgbm',
    'Ridge Classifier': 'ridge',
    'Linear Discriminant Analysis': 'lda',
    'Extreme Gradient Boosting': 'xgboost',
    'Dummy Classifier': 'dummy'
}

# Loop over each model in the compare_results DataFrame
for idx, row in compare_results.iterrows():
    model_name = row['Model']
    testing_time = row['TT (Sec)']

    # Get the PyCaret identifier for the model
    model_identifier = model_mapping.get(model_name)

    # Instead of finalizing by the model name, select the model by identifier
    if model_identifier:
        best_model = compare_models(include=[model_identifier], n_select=1)

        # Finalize the model (train it on the entire dataset)
        final_model = finalize_model(best_model)

        # Predict on train and test sets
        predictions_train = predict_model(final_model, data=X_train)
        predictions_test = predict_model(final_model, data=X_test)

        # Calculate test accuracy and F1 score
        y_pred_test = predictions_test['prediction_label']
        test_accuracy = accuracy_score(y_test, y_pred_test)
        test_f1 = f1_score(y_test, y_pred_test, average='weighted')

        # Calculate train accuracy and F1 score
        y_pred_train = predictions_train['prediction_label']
        train_accuracy = accuracy_score(y_train, y_pred_train)
        train_f1 = f1_score(y_train, y_pred_train, average='weighted')

        # Append the metrics for each model to the results DataFrame
        new_row = pd.DataFrame({
            'model': model_name,
            'train_accuracy': train_accuracy,
            'test_accuracy': test_accuracy,
            'train_f1': train_f1,
            'test_f1': test_f1,
            'TT': testing_time
        }, index=[0])

        sweeperkeeper_df = pd.concat([sweeperkeeper_df, new_row], ignore_index=True)


,Description,Value
0,Session id,42
1,Target,Class_Sweeper Keeper
2,Target type,Binary
3,Original data shape,"(91, 15)"
4,Transformed data shape,"(91, 15)"
5,Transformed train set shape,"(63, 15)"
6,Transformed test set shape,"(28, 15)"
7,Numeric features,14
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9833,0.9875,0.9500,1.0000,0.9667,0.9571,0.9632,1.3730
et,Extra Trees Classifier,0.9548,0.9875,0.8500,1.0000,0.9000,0.8748,0.8923,0.1100
nb,Naive Bayes,0.9238,0.9600,0.7500,1.0000,0.8333,0.7908,0.8201,0.0140
rf,Random Forest Classifier,0.9238,0.9875,0.7500,1.0000,0.8333,0.7908,0.8201,0.0980
knn,K Neighbors Classifier,0.9214,0.9688,0.7500,0.9000,0.8000,0.7748,0.7923,0.0250
ridge,Ridge Classifier,0.9214,0.9475,0.8000,0.9500,0.8500,0.7998,0.8173,0.0180
lda,Linear Discriminant Analysis,0.9214,0.9475,0.8000,0.9500,0.8500,0.7998,0.8173,0.0200
xgboost,Extreme Gradient Boosting,0.9071,0.9500,0.8000,0.9167,0.8133,0.7574,0.7909,0.2500
svm,SVM - Linear Kernel,0.8905,0.9675,0.8500,0.7500,0.7733,0.7160,0.7399,0.0130
lightgbm,Light Gradient Boosting Machine,0.8881,0.9200,0.8000,0.8500,0.7667,0.7034,0.7453,0.0370


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9833,0.9875,0.9500,1.0000,0.9667,0.9571,0.9632,0.0270


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,0.7905,0.7175,0.5500,0.5333,0.5067,0.4017,0.4227,0.0130


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.7310,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0120


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

In [16]:
sweeperkeeper_df

,model,train_accuracy,test_accuracy,train_f1,test_f1,TT
0,Logistic Regression,1.000000,1.000000,1.000000,1.000000,1.373
1,Extra Trees Classifier,1.000000,0.913043,1.000000,0.900725,0.110
2,Naive Bayes,0.934066,0.913043,0.930769,0.900725,0.014
3,Random Forest Classifier,1.000000,0.913043,1.000000,0.900725,0.098
4,K Neighbors Classifier,0.989011,0.956522,0.988935,0.953974,0.025
5,Ridge Classifier,0.945055,0.913043,0.943825,0.900725,0.018
6,Linear Discriminant Analysis,0.945055,0.913043,0.943825,0.900725,0.020
7,Extreme Gradient Boosting,1.000000,0.913043,1.000000,0.900725,0.250
8,SVM - Linear Kernel,0.956044,0.869565,0.956044,0.861921,0.013
9,Light Gradient Boosting Machine,1.000000,0.913043,1.000000,0.900725,0.037


## Ball-Playing Defender

In [17]:
import pandas as pd
from pycaret.classification import setup, compare_models, predict_model, finalize_model, pull
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import joblib

# Load the CSV file into a DataFrame
df = pd.read_csv('../data/players_df_sin_reco.csv')

# Select features (columns 7 to 20, 0-based index: 6 to 19)
X = df.iloc[:, 6:20]

# Select the 9th column from the end as the target
y = df.iloc[:, -9]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an empty DataFrame to store results
ballplayingdefender_df = pd.DataFrame(columns=['model', 'train_accuracy', 'test_accuracy', 'train_f1', 'test_f1', 'TT'])

# Concatenate X_train and y_train into a DataFrame for PyCaret setup
df_target = pd.concat([X_train, y_train], axis=1)

# Setup PyCaret environment
clf = setup(data=df_target, target=y.name, session_id=42)

# Compare different models and store the results
best_model = compare_models()  # This returns the top model
compare_results = pull()  # Pulls all model results as a DataFrame

# Define the mapping from model names to PyCaret identifiers
model_mapping = {
    'K Neighbors Classifier': 'knn',
    'Random Forest Classifier': 'rf',
    'Logistic Regression': 'lr',
    'Decision Tree Classifier': 'dt',
    'Ada Boost Classifier': 'ada',
    'Gradient Boosting Classifier': 'gbc',
    'Naive Bayes': 'nb',
    'SVM - Linear Kernel': 'svm',
    'Extra Trees Classifier': 'et',
    'Light Gradient Boosting Machine': 'lightgbm',
    'Ridge Classifier': 'ridge',
    'Linear Discriminant Analysis': 'lda',
    'Extreme Gradient Boosting': 'xgboost',
    'Dummy Classifier': 'dummy'
}

# Loop over each model in the compare_results DataFrame
for idx, row in compare_results.iterrows():
    model_name = row['Model']
    testing_time = row['TT (Sec)']

    # Get the PyCaret identifier for the model
    model_identifier = model_mapping.get(model_name)

    # Instead of finalizing by the model name, select the model by identifier
    if model_identifier:
        best_model = compare_models(include=[model_identifier], n_select=1)

        # Finalize the model (train it on the entire dataset)
        final_model = finalize_model(best_model)

        # Predict on train and test sets
        predictions_train = predict_model(final_model, data=X_train)
        predictions_test = predict_model(final_model, data=X_test)

        # Calculate test accuracy and F1 score
        y_pred_test = predictions_test['prediction_label']
        test_accuracy = accuracy_score(y_test, y_pred_test)
        test_f1 = f1_score(y_test, y_pred_test, average='weighted')

        # Calculate train accuracy and F1 score
        y_pred_train = predictions_train['prediction_label']
        train_accuracy = accuracy_score(y_train, y_pred_train)
        train_f1 = f1_score(y_train, y_pred_train, average='weighted')

        # Append the metrics for each model to the results DataFrame
        new_row = pd.DataFrame({
            'model': model_name,
            'train_accuracy': train_accuracy,
            'test_accuracy': test_accuracy,
            'train_f1': train_f1,
            'test_f1': test_f1,
            'TT': testing_time
        }, index=[0])

        ballplayingdefender_df = pd.concat([ballplayingdefender_df, new_row], ignore_index=True)


,Description,Value
0,Session id,42
1,Target,Class_Ball Playing Defender
2,Target type,Binary
3,Original data shape,"(91, 15)"
4,Transformed data shape,"(91, 15)"
5,Transformed train set shape,"(63, 15)"
6,Transformed test set shape,"(28, 15)"
7,Numeric features,14
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9833,1.0000,1.0000,0.9833,0.9909,0.9000,0.9000,0.0650
ridge,Ridge Classifier,0.9500,1.0000,0.9750,0.9667,0.9675,0.7667,0.7707,0.0180
lda,Linear Discriminant Analysis,0.9500,1.0000,0.9750,0.9667,0.9675,0.7667,0.7707,0.0170
knn,K Neighbors Classifier,0.9357,0.9850,0.9800,0.9467,0.9596,0.7267,0.7363,0.0250
rf,Random Forest Classifier,0.9357,1.0000,0.9800,0.9500,0.9598,0.7696,0.7730,0.1040
lightgbm,Light Gradient Boosting Machine,0.9357,1.0000,0.9550,0.9633,0.9544,0.7934,0.8070,0.0510
lr,Logistic Regression,0.9333,1.0000,0.9750,0.9500,0.9584,0.6667,0.6707,0.0200
nb,Naive Bayes,0.9333,0.9300,0.9550,0.9600,0.9546,0.8038,0.8140,0.0240
ada,Ada Boost Classifier,0.9333,0.9900,0.9750,0.9467,0.9564,0.7238,0.7340,0.0610
xgboost,Extreme Gradient Boosting,0.9214,1.0000,0.9550,0.9467,0.9453,0.7522,0.7715,0.0300


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9833,1.0000,1.0000,0.9833,0.9909,0.9000,0.9000,0.0900


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8262,0.8838,0.8900,0.8981,0.8814,0.4998,0.5214,0.0470


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.7643,0.5000,1.0000,0.7643,0.8645,0.0000,0.0000,0.0120


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

In [18]:
ballplayingdefender_df

,model,train_accuracy,test_accuracy,train_f1,test_f1,TT
0,Extra Trees Classifier,1.000000,0.956522,1.000000,0.960339,0.065
1,Ridge Classifier,0.978022,0.956522,0.977656,0.960339,0.018
2,Linear Discriminant Analysis,0.978022,0.956522,0.977656,0.960339,0.017
3,K Neighbors Classifier,0.956044,0.956522,0.954465,0.960339,0.025
4,Random Forest Classifier,1.000000,0.956522,1.000000,0.960339,0.104
5,Light Gradient Boosting Machine,1.000000,0.956522,1.000000,0.960339,0.051
6,Logistic Regression,1.000000,0.956522,1.000000,0.960339,0.020
7,Naive Bayes,0.956044,0.956522,0.956676,0.960339,0.024
8,Ada Boost Classifier,1.000000,0.956522,1.000000,0.960339,0.061
9,Extreme Gradient Boosting,1.000000,0.956522,1.000000,0.960339,0.030


## No-Nonsense Defender

In [19]:
import pandas as pd
from pycaret.classification import setup, compare_models, predict_model, finalize_model, pull
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import joblib

# Load the CSV file into a DataFrame
df = pd.read_csv('../data/players_df_sin_reco.csv')

# Select features (columns 7 to 20, 0-based index: 6 to 19)
X = df.iloc[:, 6:20]

# Select the 8th column from the end as the target
y = df.iloc[:, -8]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an empty DataFrame to store results
nononsensedefender_df = pd.DataFrame(columns=['model', 'train_accuracy', 'test_accuracy', 'train_f1', 'test_f1', 'TT'])

# Concatenate X_train and y_train into a DataFrame for PyCaret setup
df_target = pd.concat([X_train, y_train], axis=1)

# Setup PyCaret environment
clf = setup(data=df_target, target=y.name, session_id=42)

# Compare different models and store the results
best_model = compare_models()  # This returns the top model
compare_results = pull()  # Pulls all model results as a DataFrame

# Define the mapping from model names to PyCaret identifiers
model_mapping = {
    'K Neighbors Classifier': 'knn',
    'Random Forest Classifier': 'rf',
    'Logistic Regression': 'lr',
    'Decision Tree Classifier': 'dt',
    'Ada Boost Classifier': 'ada',
    'Gradient Boosting Classifier': 'gbc',
    'Naive Bayes': 'nb',
    'SVM - Linear Kernel': 'svm',
    'Extra Trees Classifier': 'et',
    'Light Gradient Boosting Machine': 'lightgbm',
    'Ridge Classifier': 'ridge',
    'Linear Discriminant Analysis': 'lda',
    'Extreme Gradient Boosting': 'xgboost',
    'Dummy Classifier': 'dummy'
}

# Loop over each model in the compare_results DataFrame
for idx, row in compare_results.iterrows():
    model_name = row['Model']
    testing_time = row['TT (Sec)']

    # Get the PyCaret identifier for the model
    model_identifier = model_mapping.get(model_name)

    # Instead of finalizing by the model name, select the model by identifier
    if model_identifier:
        best_model = compare_models(include=[model_identifier], n_select=1)

        # Finalize the model (train it on the entire dataset)
        final_model = finalize_model(best_model)

        # Predict on train and test sets
        predictions_train = predict_model(final_model, data=X_train)
        predictions_test = predict_model(final_model, data=X_test)

        # Calculate test accuracy and F1 score
        y_pred_test = predictions_test['prediction_label']
        test_accuracy = accuracy_score(y_test, y_pred_test)
        test_f1 = f1_score(y_test, y_pred_test, average='weighted')

        # Calculate train accuracy and F1 score
        y_pred_train = predictions_train['prediction_label']
        train_accuracy = accuracy_score(y_train, y_pred_train)
        train_f1 = f1_score(y_train, y_pred_train, average='weighted')

        # Append the metrics for each model to the results DataFrame
        new_row = pd.DataFrame({
            'model': model_name,
            'train_accuracy': train_accuracy,
            'test_accuracy': test_accuracy,
            'train_f1': train_f1,
            'test_f1': test_f1,
            'TT': testing_time
        }, index=[0])

        nononsensedefender_df = pd.concat([nononsensedefender_df, new_row], ignore_index=True)


,Description,Value
0,Session id,42
1,Target,Class_No Nonsense Defender
2,Target type,Binary
3,Original data shape,"(91, 15)"
4,Transformed data shape,"(91, 15)"
5,Transformed train set shape,"(63, 15)"
6,Transformed test set shape,"(28, 15)"
7,Numeric features,14
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9690,1.0000,0.9800,0.9800,0.9778,0.9267,0.9363,0.0300
et,Extra Trees Classifier,0.9381,1.0000,0.9750,0.9433,0.9544,0.8451,0.8660,0.0790
lightgbm,Light Gradient Boosting Machine,0.9381,0.9575,0.9750,0.9433,0.9544,0.8451,0.8660,0.0360
ada,Ada Boost Classifier,0.9238,0.9750,0.9550,0.9433,0.9433,0.8147,0.8391,0.0660
xgboost,Extreme Gradient Boosting,0.9238,0.9792,0.9750,0.9183,0.9437,0.8148,0.8327,0.0210
knn,K Neighbors Classifier,0.9167,0.9875,0.9750,0.9200,0.9413,0.7952,0.8237,0.0280
ridge,Ridge Classifier,0.9048,0.9875,0.9550,0.9183,0.9326,0.7677,0.7891,0.0170
lda,Linear Discriminant Analysis,0.9048,0.9875,0.9550,0.9183,0.9326,0.7677,0.7891,0.0130
rf,Random Forest Classifier,0.8905,0.9900,0.9300,0.9233,0.9179,0.7385,0.7730,0.1220
nb,Naive Bayes,0.8738,0.9238,0.9050,0.9250,0.9062,0.6921,0.7083,0.0140


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9690,1.0000,0.9800,0.9800,0.9778,0.9267,0.9363,0.0410


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,0.7929,0.7675,0.8350,0.8733,0.8397,0.5317,0.5755,0.0160


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.6667,0.5000,1.0000,0.6667,0.7994,0.0000,0.0000,0.0120


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

In [20]:
nononsensedefender_df

,model,train_accuracy,test_accuracy,train_f1,test_f1,TT
0,Logistic Regression,1.000000,0.956522,1.000000,0.954694,0.030
1,Extra Trees Classifier,1.000000,0.826087,1.000000,0.808924,0.079
2,Light Gradient Boosting Machine,1.000000,0.826087,1.000000,0.808924,0.036
3,Ada Boost Classifier,1.000000,0.869565,1.000000,0.846632,0.066
4,Extreme Gradient Boosting,1.000000,0.869565,1.000000,0.846632,0.021
5,K Neighbors Classifier,0.901099,0.782609,0.897531,0.773469,0.028
6,Ridge Classifier,0.956044,0.869565,0.955672,0.864081,0.017
7,Linear Discriminant Analysis,0.956044,0.869565,0.955672,0.864081,0.013
8,Random Forest Classifier,1.000000,0.826087,1.000000,0.808924,0.122
9,Naive Bayes,0.912088,0.782609,0.912717,0.773469,0.014


## Full-Back

In [21]:
import pandas as pd
from pycaret.classification import setup, compare_models, predict_model, finalize_model, pull
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import joblib

# Load the CSV file into a DataFrame
df = pd.read_csv('../data/players_df_sin_reco.csv')

# Select features (columns 7 to 20, 0-based index: 6 to 19)
X = df.iloc[:, 6:20]

# Select the 7th column from the end as the target
y = df.iloc[:, -7]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an empty DataFrame to store results
fullback_df = pd.DataFrame(columns=['model', 'train_accuracy', 'test_accuracy', 'train_f1', 'test_f1', 'TT'])

# Concatenate X_train and y_train into a DataFrame for PyCaret setup
df_target = pd.concat([X_train, y_train], axis=1)

# Setup PyCaret environment
clf = setup(data=df_target, target=y.name, session_id=42)

# Compare different models and store the results
best_model = compare_models()  # This returns the top model
compare_results = pull()  # Pulls all model results as a DataFrame

# Define the mapping from model names to PyCaret identifiers
model_mapping = {
    'K Neighbors Classifier': 'knn',
    'Random Forest Classifier': 'rf',
    'Logistic Regression': 'lr',
    'Decision Tree Classifier': 'dt',
    'Ada Boost Classifier': 'ada',
    'Gradient Boosting Classifier': 'gbc',
    'Naive Bayes': 'nb',
    'SVM - Linear Kernel': 'svm',
    'Extra Trees Classifier': 'et',
    'Light Gradient Boosting Machine': 'lightgbm',
    'Ridge Classifier': 'ridge',
    'Linear Discriminant Analysis': 'lda',
    'Extreme Gradient Boosting': 'xgboost',
    'Dummy Classifier': 'dummy'
}

# Loop over each model in the compare_results DataFrame
for idx, row in compare_results.iterrows():
    model_name = row['Model']
    testing_time = row['TT (Sec)']

    # Get the PyCaret identifier for the model
    model_identifier = model_mapping.get(model_name)

    # Instead of finalizing by the model name, select the model by identifier
    if model_identifier:
        best_model = compare_models(include=[model_identifier], n_select=1)

        # Finalize the model (train it on the entire dataset)
        final_model = finalize_model(best_model)

        # Predict on train and test sets
        predictions_train = predict_model(final_model, data=X_train)
        predictions_test = predict_model(final_model, data=X_test)

        # Calculate test accuracy and F1 score
        y_pred_test = predictions_test['prediction_label']
        test_accuracy = accuracy_score(y_test, y_pred_test)
        test_f1 = f1_score(y_test, y_pred_test, average='weighted')

        # Calculate train accuracy and F1 score
        y_pred_train = predictions_train['prediction_label']
        train_accuracy = accuracy_score(y_train, y_pred_train)
        train_f1 = f1_score(y_train, y_pred_train, average='weighted')

        # Append the metrics for each model to the results DataFrame
        new_row = pd.DataFrame({
            'model': model_name,
            'train_accuracy': train_accuracy,
            'test_accuracy': test_accuracy,
            'train_f1': train_f1,
            'test_f1': test_f1,
            'TT': testing_time
        }, index=[0])

        fullback_df = pd.concat([fullback_df, new_row], ignore_index=True)


,Description,Value
0,Session id,42
1,Target,Class_Full Back
2,Target type,Binary
3,Original data shape,"(91, 15)"
4,Transformed data shape,"(91, 15)"
5,Transformed train set shape,"(63, 15)"
6,Transformed test set shape,"(28, 15)"
7,Numeric features,14
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9214,0.9675,0.9550,0.9500,0.9455,0.7951,0.8083,0.0340
ridge,Ridge Classifier,0.8881,0.9550,0.9550,0.9033,0.9211,0.7236,0.7613,0.0190
et,Extra Trees Classifier,0.8738,0.9500,0.9550,0.8914,0.9135,0.6648,0.6967,0.1080
lda,Linear Discriminant Analysis,0.8714,0.9550,0.9300,0.8983,0.9072,0.6915,0.7230,0.0130
knn,K Neighbors Classifier,0.8690,0.9938,0.9550,0.8933,0.9124,0.6505,0.6702,0.0220
rf,Random Forest Classifier,0.8405,0.9438,0.9550,0.8648,0.8957,0.5505,0.5702,0.0910
ada,Ada Boost Classifier,0.8405,0.8875,0.9300,0.8650,0.8894,0.5948,0.6168,0.0580
nb,Naive Bayes,0.8381,0.8900,0.8800,0.8767,0.8719,0.6427,0.6641,0.0160
xgboost,Extreme Gradient Boosting,0.8381,0.9025,0.9300,0.8717,0.8894,0.5772,0.5965,0.0290
lightgbm,Light Gradient Boosting Machine,0.8071,0.9125,0.9050,0.8514,0.8640,0.4934,0.5070,0.0310


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9214,0.9675,0.9550,0.9500,0.9455,0.7951,0.8083,0.0350


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
svm,SVM - Linear Kernel,0.7214,0.9275,0.6500,0.8633,0.6992,0.5114,0.5646,0.0150


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.6976,0.5000,1.0000,0.6976,0.8209,0.0000,0.0000,0.0150


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

In [22]:
fullback_df

,model,train_accuracy,test_accuracy,train_f1,test_f1,TT
0,Logistic Regression,1.000000,0.956522,1.000000,0.952704,0.034
1,Ridge Classifier,0.967033,0.913043,0.966441,0.913043,0.019
2,Extra Trees Classifier,1.000000,0.913043,1.000000,0.913043,0.108
3,Linear Discriminant Analysis,0.967033,0.913043,0.966441,0.913043,0.013
4,K Neighbors Classifier,0.901099,0.913043,0.899323,0.913043,0.022
5,Random Forest Classifier,1.000000,0.913043,1.000000,0.913043,0.091
6,Ada Boost Classifier,1.000000,0.956522,1.000000,0.952704,0.058
7,Naive Bayes,0.912088,0.869565,0.912088,0.877210,0.016
8,Extreme Gradient Boosting,1.000000,0.913043,1.000000,0.913043,0.029
9,Light Gradient Boosting Machine,1.000000,0.956522,1.000000,0.952704,0.031


## All-Action Midfielder

In [23]:
import pandas as pd
from pycaret.classification import setup, compare_models, predict_model, finalize_model, pull
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import joblib

# Load the CSV file into a DataFrame
df = pd.read_csv('../data/players_df_sin_reco.csv')

# Select features (columns 7 to 20, 0-based index: 6 to 19)
X = df.iloc[:, 6:20]

# Select the 6th column from the end as the target
y = df.iloc[:, -6]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an empty DataFrame to store results
allactionmidfielder_df = pd.DataFrame(columns=['model', 'train_accuracy', 'test_accuracy', 'train_f1', 'test_f1', 'TT'])

# Concatenate X_train and y_train into a DataFrame for PyCaret setup
df_target = pd.concat([X_train, y_train], axis=1)

# Setup PyCaret environment
clf = setup(data=df_target, target=y.name, session_id=42)

# Compare different models and store the results
best_model = compare_models()  # This returns the top model
compare_results = pull()  # Pulls all model results as a DataFrame

# Define the mapping from model names to PyCaret identifiers
model_mapping = {
    'K Neighbors Classifier': 'knn',
    'Random Forest Classifier': 'rf',
    'Logistic Regression': 'lr',
    'Decision Tree Classifier': 'dt',
    'Ada Boost Classifier': 'ada',
    'Gradient Boosting Classifier': 'gbc',
    'Naive Bayes': 'nb',
    'SVM - Linear Kernel': 'svm',
    'Extra Trees Classifier': 'et',
    'Light Gradient Boosting Machine': 'lightgbm',
    'Ridge Classifier': 'ridge',
    'Linear Discriminant Analysis': 'lda',
    'Extreme Gradient Boosting': 'xgboost',
    'Dummy Classifier': 'dummy'
}

# Loop over each model in the compare_results DataFrame
for idx, row in compare_results.iterrows():
    model_name = row['Model']
    testing_time = row['TT (Sec)']

    # Get the PyCaret identifier for the model
    model_identifier = model_mapping.get(model_name)

    # Instead of finalizing by the model name, select the model by identifier
    if model_identifier:
        best_model = compare_models(include=[model_identifier], n_select=1)

        # Finalize the model (train it on the entire dataset)
        final_model = finalize_model(best_model)

        # Predict on train and test sets
        predictions_train = predict_model(final_model, data=X_train)
        predictions_test = predict_model(final_model, data=X_test)

        # Calculate test accuracy and F1 score
        y_pred_test = predictions_test['prediction_label']
        test_accuracy = accuracy_score(y_test, y_pred_test)
        test_f1 = f1_score(y_test, y_pred_test, average='weighted')

        # Calculate train accuracy and F1 score
        y_pred_train = predictions_train['prediction_label']
        train_accuracy = accuracy_score(y_train, y_pred_train)
        train_f1 = f1_score(y_train, y_pred_train, average='weighted')

        # Append the metrics for each model to the results DataFrame
        new_row = pd.DataFrame({
            'model': model_name,
            'train_accuracy': train_accuracy,
            'test_accuracy': test_accuracy,
            'train_f1': train_f1,
            'test_f1': test_f1,
            'TT': testing_time
        }, index=[0])

        allactionmidfielder_df = pd.concat([allactionmidfielder_df, new_row], ignore_index=True)


,Description,Value
0,Session id,42
1,Target,Class_All Action Midfielder
2,Target type,Binary
3,Original data shape,"(91, 15)"
4,Transformed data shape,"(91, 15)"
5,Transformed train set shape,"(63, 15)"
6,Transformed test set shape,"(28, 15)"
7,Numeric features,14
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9381,1.0000,0.9550,0.9600,0.9524,0.8559,0.8745,0.0330
knn,K Neighbors Classifier,0.9333,0.9375,1.0000,0.9267,0.9578,0.8143,0.8265,0.0230
ridge,Ridge Classifier,0.9048,0.9792,0.9800,0.9000,0.9333,0.7677,0.7990,0.0170
gbc,Gradient Boosting Classifier,0.9048,0.9317,0.9050,0.9500,0.9246,0.7916,0.7980,0.0540
lda,Linear Discriminant Analysis,0.9048,0.9792,0.9800,0.9000,0.9333,0.7677,0.7990,0.0130
nb,Naive Bayes,0.8905,0.9192,0.8850,0.9550,0.9103,0.7670,0.7887,0.0140
dt,Decision Tree Classifier,0.8881,0.8775,0.9050,0.9300,0.9135,0.7487,0.7613,0.0150
xgboost,Extreme Gradient Boosting,0.8738,0.9292,0.9300,0.8900,0.9056,0.7077,0.7294,0.0240
rf,Random Forest Classifier,0.8714,0.9667,0.9500,0.8700,0.9056,0.6952,0.7196,0.1050
lightgbm,Light Gradient Boosting Machine,0.8690,0.9750,0.9800,0.8733,0.9156,0.6410,0.6628,0.0450


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9381,1.0000,0.9550,0.9600,0.9524,0.8559,0.8745,0.0330


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
svm,SVM - Linear Kernel,0.8548,0.9625,0.9050,0.8867,0.8873,0.6772,0.7065,0.0160


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.6667,0.5000,1.0000,0.6667,0.7994,0.0000,0.0000,0.0120


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

In [24]:
allactionmidfielder_df

,model,train_accuracy,test_accuracy,train_f1,test_f1,TT
0,Logistic Regression,1.000000,0.869565,1.000000,0.875049,0.033
1,K Neighbors Classifier,0.934066,0.869565,0.932758,0.875049,0.023
2,Ridge Classifier,0.945055,0.869565,0.944268,0.875049,0.017
3,Gradient Boosting Classifier,1.000000,0.913043,1.000000,0.913043,0.054
4,Linear Discriminant Analysis,0.956044,0.869565,0.955641,0.875049,0.013
5,Naive Bayes,0.923077,0.826087,0.923390,0.826087,0.014
6,Decision Tree Classifier,1.000000,0.782609,1.000000,0.802936,0.015
7,Extreme Gradient Boosting,1.000000,0.869565,1.000000,0.861921,0.024
8,Random Forest Classifier,1.000000,0.869565,1.000000,0.861921,0.105
9,Light Gradient Boosting Machine,1.000000,0.869565,1.000000,0.861921,0.045


## Midfield Playmaker

In [25]:
import pandas as pd
from pycaret.classification import setup, compare_models, predict_model, finalize_model, pull
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import joblib

# Load the CSV file into a DataFrame
df = pd.read_csv('../data/players_df_sin_reco.csv')

# Select features (columns 7 to 20, 0-based index: 6 to 19)
X = df.iloc[:, 6:20]

# Select the 5thth column from the end as the target
y = df.iloc[:, -5]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an empty DataFrame to store results
midfieldplaymaker_df = pd.DataFrame(columns=['model', 'train_accuracy', 'test_accuracy', 'train_f1', 'test_f1', 'TT'])

# Concatenate X_train and y_train into a DataFrame for PyCaret setup
df_target = pd.concat([X_train, y_train], axis=1)

# Setup PyCaret environment
clf = setup(data=df_target, target=y.name, session_id=42)

# Compare different models and store the results
best_model = compare_models()  # This returns the top model
compare_results = pull()  # Pulls all model results as a DataFrame

# Define the mapping from model names to PyCaret identifiers
model_mapping = {
    'K Neighbors Classifier': 'knn',
    'Random Forest Classifier': 'rf',
    'Logistic Regression': 'lr',
    'Decision Tree Classifier': 'dt',
    'Ada Boost Classifier': 'ada',
    'Gradient Boosting Classifier': 'gbc',
    'Naive Bayes': 'nb',
    'SVM - Linear Kernel': 'svm',
    'Extra Trees Classifier': 'et',
    'Light Gradient Boosting Machine': 'lightgbm',
    'Ridge Classifier': 'ridge',
    'Linear Discriminant Analysis': 'lda',
    'Extreme Gradient Boosting': 'xgboost',
    'Dummy Classifier': 'dummy'
}

# Loop over each model in the compare_results DataFrame
for idx, row in compare_results.iterrows():
    model_name = row['Model']
    testing_time = row['TT (Sec)']

    # Get the PyCaret identifier for the model
    model_identifier = model_mapping.get(model_name)

    # Instead of finalizing by the model name, select the model by identifier
    if model_identifier:
        best_model = compare_models(include=[model_identifier], n_select=1)

        # Finalize the model (train it on the entire dataset)
        final_model = finalize_model(best_model)

        # Predict on train and test sets
        predictions_train = predict_model(final_model, data=X_train)
        predictions_test = predict_model(final_model, data=X_test)

        # Calculate test accuracy and F1 score
        y_pred_test = predictions_test['prediction_label']
        test_accuracy = accuracy_score(y_test, y_pred_test)
        test_f1 = f1_score(y_test, y_pred_test, average='weighted')

        # Calculate train accuracy and F1 score
        y_pred_train = predictions_train['prediction_label']
        train_accuracy = accuracy_score(y_train, y_pred_train)
        train_f1 = f1_score(y_train, y_pred_train, average='weighted')

        # Append the metrics for each model to the results DataFrame
        new_row = pd.DataFrame({
            'model': model_name,
            'train_accuracy': train_accuracy,
            'test_accuracy': test_accuracy,
            'train_f1': train_f1,
            'test_f1': test_f1,
            'TT': testing_time
        }, index=[0])

        midfieldplaymaker_df = pd.concat([midfieldplaymaker_df, new_row], ignore_index=True)


,Description,Value
0,Session id,42
1,Target,Class_Midfield Playmaker
2,Target type,Binary
3,Original data shape,"(91, 15)"
4,Transformed data shape,"(91, 15)"
5,Transformed train set shape,"(63, 15)"
6,Transformed test set shape,"(28, 15)"
7,Numeric features,14
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9571,0.9800,0.9633,0.9833,0.9707,0.8872,0.9021,0.0260
et,Extra Trees Classifier,0.9571,0.9733,0.9833,0.9714,0.9742,0.8588,0.8645,0.0850
xgboost,Extreme Gradient Boosting,0.9429,0.9633,0.9633,0.9714,0.9631,0.8284,0.8376,0.0290
lightgbm,Light Gradient Boosting Machine,0.9405,0.9833,0.9833,0.9548,0.9652,0.7588,0.7645,0.0340
ada,Ada Boost Classifier,0.9381,0.9483,0.9633,0.9667,0.9616,0.7748,0.7923,0.0540
nb,Naive Bayes,0.9262,0.9233,0.9433,0.9667,0.9525,0.7924,0.8020,0.0220
lda,Linear Discriminant Analysis,0.9262,0.9900,0.9633,0.9467,0.9527,0.7476,0.7591,0.0150
knn,K Neighbors Classifier,0.9238,0.9533,0.9633,0.9500,0.9540,0.6993,0.7111,0.0250
rf,Random Forest Classifier,0.9238,0.9633,0.9633,0.9514,0.9542,0.7388,0.7445,0.0850
svm,SVM - Linear Kernel,0.9095,0.9633,0.9833,0.9214,0.9470,0.6176,0.6291,0.0160


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9571,0.9800,0.9633,0.9833,0.9707,0.8872,0.9021,0.0270


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,0.8929,0.8217,0.9433,0.9348,0.9340,0.6084,0.6176,0.0230


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.8119,0.5000,1.0000,0.8119,0.8953,0.0000,0.0000,0.0160


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

In [26]:
midfieldplaymaker_df

,model,train_accuracy,test_accuracy,train_f1,test_f1,TT
0,Logistic Regression,1.000000,1.000000,1.000000,1.000000,0.026
1,Extra Trees Classifier,1.000000,0.956522,1.000000,0.949781,0.085
2,Extreme Gradient Boosting,0.989011,0.956522,0.989131,0.949781,0.029
3,Light Gradient Boosting Machine,1.000000,0.956522,1.000000,0.949781,0.034
4,Ada Boost Classifier,1.000000,0.956522,1.000000,0.949781,0.054
5,Naive Bayes,0.945055,1.000000,0.946714,1.000000,0.022
6,Linear Discriminant Analysis,0.989011,1.000000,0.988881,1.000000,0.015
7,K Neighbors Classifier,0.934066,0.956522,0.930538,0.949781,0.025
8,Random Forest Classifier,1.000000,0.956522,1.000000,0.949781,0.085
9,SVM - Linear Kernel,0.945055,1.000000,0.947616,1.000000,0.016


## Traditional Winger

In [27]:
import pandas as pd
from pycaret.classification import setup, compare_models, predict_model, finalize_model, pull
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import joblib

# Load the CSV file into a DataFrame
df = pd.read_csv('../data/players_df_sin_reco.csv')

# Select features (columns 7 to 20, 0-based index: 6 to 19)
X = df.iloc[:, 6:20]

# Select the 4th column from the end as the target
y = df.iloc[:, -4]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an empty DataFrame to store results
traditionalwinger_df = pd.DataFrame(columns=['model', 'train_accuracy', 'test_accuracy', 'train_f1', 'test_f1', 'TT'])

# Concatenate X_train and y_train into a DataFrame for PyCaret setup
df_target = pd.concat([X_train, y_train], axis=1)

# Setup PyCaret environment
clf = setup(data=df_target, target=y.name, session_id=42)

# Compare different models and store the results
best_model = compare_models()  # This returns the top model
compare_results = pull()  # Pulls all model results as a DataFrame

# Define the mapping from model names to PyCaret identifiers
model_mapping = {
    'K Neighbors Classifier': 'knn',
    'Random Forest Classifier': 'rf',
    'Logistic Regression': 'lr',
    'Decision Tree Classifier': 'dt',
    'Ada Boost Classifier': 'ada',
    'Gradient Boosting Classifier': 'gbc',
    'Naive Bayes': 'nb',
    'SVM - Linear Kernel': 'svm',
    'Extra Trees Classifier': 'et',
    'Light Gradient Boosting Machine': 'lightgbm',
    'Ridge Classifier': 'ridge',
    'Linear Discriminant Analysis': 'lda',
    'Extreme Gradient Boosting': 'xgboost',
    'Dummy Classifier': 'dummy'
}

# Loop over each model in the compare_results DataFrame
for idx, row in compare_results.iterrows():
    model_name = row['Model']
    testing_time = row['TT (Sec)']

    # Get the PyCaret identifier for the model
    model_identifier = model_mapping.get(model_name)

    # Instead of finalizing by the model name, select the model by identifier
    if model_identifier:
        best_model = compare_models(include=[model_identifier], n_select=1)

        # Finalize the model (train it on the entire dataset)
        final_model = finalize_model(best_model)

        # Predict on train and test sets
        predictions_train = predict_model(final_model, data=X_train)
        predictions_test = predict_model(final_model, data=X_test)

        # Calculate test accuracy and F1 score
        y_pred_test = predictions_test['prediction_label']
        test_accuracy = accuracy_score(y_test, y_pred_test)
        test_f1 = f1_score(y_test, y_pred_test, average='weighted')

        # Calculate train accuracy and F1 score
        y_pred_train = predictions_train['prediction_label']
        train_accuracy = accuracy_score(y_train, y_pred_train)
        train_f1 = f1_score(y_train, y_pred_train, average='weighted')

        # Append the metrics for each model to the results DataFrame
        new_row = pd.DataFrame({
            'model': model_name,
            'train_accuracy': train_accuracy,
            'test_accuracy': test_accuracy,
            'train_f1': train_f1,
            'test_f1': test_f1,
            'TT': testing_time
        }, index=[0])

        traditionalwinger_df = pd.concat([traditionalwinger_df, new_row], ignore_index=True)


,Description,Value
0,Session id,42
1,Target,Class_Traditional Winger
2,Target type,Binary
3,Original data shape,"(91, 15)"
4,Transformed data shape,"(91, 15)"
5,Transformed train set shape,"(63, 15)"
6,Transformed test set shape,"(28, 15)"
7,Numeric features,14
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9548,1.0000,0.9667,0.9417,0.9457,0.9082,0.9187,0.0390
ridge,Ridge Classifier,0.8738,0.9583,0.7833,0.9167,0.8133,0.7227,0.7544,0.0130
lda,Linear Discriminant Analysis,0.8738,0.9583,0.7833,0.9167,0.8133,0.7227,0.7544,0.0140
xgboost,Extreme Gradient Boosting,0.8714,0.9292,0.8667,0.8333,0.8300,0.7305,0.7506,0.0360
ada,Ada Boost Classifier,0.8524,0.9625,0.9000,0.7983,0.8195,0.7057,0.7363,0.0470
gbc,Gradient Boosting Classifier,0.8262,0.9375,0.7000,0.8667,0.7333,0.6145,0.6535,0.0570
nb,Naive Bayes,0.8238,0.8896,0.8667,0.7733,0.7852,0.6472,0.6843,0.0170
rf,Random Forest Classifier,0.8119,0.9042,0.7500,0.8233,0.7538,0.6104,0.6395,0.0960
et,Extra Trees Classifier,0.8095,0.9146,0.7333,0.7817,0.7229,0.5871,0.6162,0.1120
dt,Decision Tree Classifier,0.8048,0.7833,0.7167,0.7833,0.7133,0.5698,0.6021,0.0160


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9548,1.0000,0.9667,0.9417,0.9457,0.9082,0.9187,0.0400


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.7143,0.8646,0.5167,0.6150,0.5162,0.3460,0.3828,0.0280


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.6381,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0150


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

In [28]:
traditionalwinger_df

,model,train_accuracy,test_accuracy,train_f1,test_f1,TT
0,Logistic Regression,1.000000,1.000000,1.000000,1.000000,0.039
1,Ridge Classifier,0.967033,1.000000,0.966920,1.000000,0.013
2,Linear Discriminant Analysis,0.967033,1.000000,0.966920,1.000000,0.014
3,Extreme Gradient Boosting,1.000000,0.956522,1.000000,0.956356,0.036
4,Ada Boost Classifier,1.000000,0.956522,1.000000,0.956356,0.047
5,Gradient Boosting Classifier,1.000000,0.913043,1.000000,0.913043,0.057
6,Naive Bayes,0.835165,0.913043,0.838123,0.912040,0.017
7,Random Forest Classifier,1.000000,0.956522,1.000000,0.956522,0.096
8,Extra Trees Classifier,1.000000,0.913043,1.000000,0.912714,0.112
9,Decision Tree Classifier,1.000000,0.869565,1.000000,0.869068,0.016


## Inverted Winger

In [29]:
import pandas as pd
from pycaret.classification import setup, compare_models, predict_model, finalize_model, pull
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import joblib

# Load the CSV file into a DataFrame
df = pd.read_csv('../data/players_df_sin_reco.csv')

# Select features (columns 7 to 20, 0-based index: 6 to 19)
X = df.iloc[:, 6:20]

# Select the 3rd column from the end as the target
y = df.iloc[:, -3]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an empty DataFrame to store results
invertedwinger_df = pd.DataFrame(columns=['model', 'train_accuracy', 'test_accuracy', 'train_f1', 'test_f1', 'TT'])

# Concatenate X_train and y_train into a DataFrame for PyCaret setup
df_target = pd.concat([X_train, y_train], axis=1)

# Setup PyCaret environment
clf = setup(data=df_target, target=y.name, session_id=42)

# Compare different models and store the results
best_model = compare_models()  # This returns the top model
compare_results = pull()  # Pulls all model results as a DataFrame

# Define the mapping from model names to PyCaret identifiers
model_mapping = {
    'K Neighbors Classifier': 'knn',
    'Random Forest Classifier': 'rf',
    'Logistic Regression': 'lr',
    'Decision Tree Classifier': 'dt',
    'Ada Boost Classifier': 'ada',
    'Gradient Boosting Classifier': 'gbc',
    'Naive Bayes': 'nb',
    'SVM - Linear Kernel': 'svm',
    'Extra Trees Classifier': 'et',
    'Light Gradient Boosting Machine': 'lightgbm',
    'Ridge Classifier': 'ridge',
    'Linear Discriminant Analysis': 'lda',
    'Extreme Gradient Boosting': 'xgboost',
    'Dummy Classifier': 'dummy'
}

# Loop over each model in the compare_results DataFrame
for idx, row in compare_results.iterrows():
    model_name = row['Model']
    testing_time = row['TT (Sec)']

    # Get the PyCaret identifier for the model
    model_identifier = model_mapping.get(model_name)

    # Instead of finalizing by the model name, select the model by identifier
    if model_identifier:
        best_model = compare_models(include=[model_identifier], n_select=1)

        # Finalize the model (train it on the entire dataset)
        final_model = finalize_model(best_model)

        # Predict on train and test sets
        predictions_train = predict_model(final_model, data=X_train)
        predictions_test = predict_model(final_model, data=X_test)

        # Calculate test accuracy and F1 score
        y_pred_test = predictions_test['prediction_label']
        test_accuracy = accuracy_score(y_test, y_pred_test)
        test_f1 = f1_score(y_test, y_pred_test, average='weighted')

        # Calculate train accuracy and F1 score
        y_pred_train = predictions_train['prediction_label']
        train_accuracy = accuracy_score(y_train, y_pred_train)
        train_f1 = f1_score(y_train, y_pred_train, average='weighted')

        # Append the metrics for each model to the results DataFrame
        new_row = pd.DataFrame({
            'model': model_name,
            'train_accuracy': train_accuracy,
            'test_accuracy': test_accuracy,
            'train_f1': train_f1,
            'test_f1': test_f1,
            'TT': testing_time
        }, index=[0])

        invertedwinger_df = pd.concat([invertedwinger_df, new_row], ignore_index=True)


,Description,Value
0,Session id,42
1,Target,Class_Inverted Winger
2,Target type,Binary
3,Original data shape,"(91, 15)"
4,Transformed data shape,"(91, 15)"
5,Transformed train set shape,"(63, 15)"
6,Transformed test set shape,"(28, 15)"
7,Numeric features,14
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9690,0.9889,0.9667,0.9750,0.9657,0.9387,0.9457,0.0310
ridge,Ridge Classifier,0.9524,0.9889,0.9667,0.9417,0.9524,0.9053,0.9083,0.0150
lda,Linear Discriminant Analysis,0.9524,0.9889,0.9667,0.9417,0.9524,0.9053,0.9083,0.0150
et,Extra Trees Classifier,0.9500,0.9833,0.9000,1.0000,0.9400,0.9000,0.9121,0.0760
rf,Random Forest Classifier,0.9190,0.9444,0.9000,0.9500,0.9114,0.8387,0.8578,0.0890
gbc,Gradient Boosting Classifier,0.9190,0.9556,0.9000,0.9250,0.9057,0.8362,0.8437,0.0650
knn,K Neighbors Classifier,0.9167,0.9556,0.8667,0.9750,0.8957,0.8333,0.8569,0.0300
ada,Ada Boost Classifier,0.9048,0.9778,0.9333,0.8917,0.9038,0.8082,0.8228,0.0490
xgboost,Extreme Gradient Boosting,0.8762,0.9500,0.9000,0.8517,0.8674,0.7545,0.7671,0.0280
nb,Naive Bayes,0.8690,0.9000,0.8333,0.8917,0.8524,0.7362,0.7478,0.0160


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9690,0.9889,0.9667,0.9750,0.9657,0.9387,0.9457,0.0440


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,0.7762,0.7750,0.7333,0.7833,0.7448,0.5474,0.5629,0.0220


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.5214,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0210


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

In [30]:
invertedwinger_df

,model,train_accuracy,test_accuracy,train_f1,test_f1,TT
0,Logistic Regression,1.000000,0.869565,1.000000,0.870062,0.031
1,Ridge Classifier,0.978022,0.869565,0.978038,0.868548,0.015
2,Linear Discriminant Analysis,0.978022,0.869565,0.978038,0.868548,0.015
3,Extra Trees Classifier,1.000000,0.869565,1.000000,0.870062,0.076
4,Random Forest Classifier,1.000000,0.826087,1.000000,0.826087,0.089
5,Gradient Boosting Classifier,1.000000,0.869565,1.000000,0.869565,0.065
6,K Neighbors Classifier,0.934066,0.826087,0.934114,0.826087,0.030
7,Ada Boost Classifier,1.000000,0.913043,1.000000,0.913373,0.049
8,Extreme Gradient Boosting,1.000000,0.869565,1.000000,0.870062,0.028
9,Naive Bayes,0.879121,0.782609,0.879033,0.780914,0.016


## Goal Poacher

In [31]:
import pandas as pd
from pycaret.classification import setup, compare_models, predict_model, finalize_model, pull
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import joblib

# Load the CSV file into a DataFrame
df = pd.read_csv('../data/players_df_sin_reco.csv')

# Select features (columns 7 to 20, 0-based index: 6 to 19)
X = df.iloc[:, 6:20]

# Select the 2nd column from the end as the target
y = df.iloc[:, -2]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an empty DataFrame to store results
goalpoacher_df = pd.DataFrame(columns=['model', 'train_accuracy', 'test_accuracy', 'train_f1', 'test_f1', 'TT'])

# Concatenate X_train and y_train into a DataFrame for PyCaret setup
df_target = pd.concat([X_train, y_train], axis=1)

# Setup PyCaret environment
clf = setup(data=df_target, target=y.name, session_id=42)

# Compare different models and store the results
best_model = compare_models()  # This returns the top model
compare_results = pull()  # Pulls all model results as a DataFrame

# Define the mapping from model names to PyCaret identifiers
model_mapping = {
    'K Neighbors Classifier': 'knn',
    'Random Forest Classifier': 'rf',
    'Logistic Regression': 'lr',
    'Decision Tree Classifier': 'dt',
    'Ada Boost Classifier': 'ada',
    'Gradient Boosting Classifier': 'gbc',
    'Naive Bayes': 'nb',
    'SVM - Linear Kernel': 'svm',
    'Extra Trees Classifier': 'et',
    'Light Gradient Boosting Machine': 'lightgbm',
    'Ridge Classifier': 'ridge',
    'Linear Discriminant Analysis': 'lda',
    'Extreme Gradient Boosting': 'xgboost',
    'Dummy Classifier': 'dummy'
}

# Loop over each model in the compare_results DataFrame
for idx, row in compare_results.iterrows():
    model_name = row['Model']
    testing_time = row['TT (Sec)']

    # Get the PyCaret identifier for the model
    model_identifier = model_mapping.get(model_name)

    # Instead of finalizing by the model name, select the model by identifier
    if model_identifier:
        best_model = compare_models(include=[model_identifier], n_select=1)

        # Finalize the model (train it on the entire dataset)
        final_model = finalize_model(best_model)

        # Predict on train and test sets
        predictions_train = predict_model(final_model, data=X_train)
        predictions_test = predict_model(final_model, data=X_test)

        # Calculate test accuracy and F1 score
        y_pred_test = predictions_test['prediction_label']
        test_accuracy = accuracy_score(y_test, y_pred_test)
        test_f1 = f1_score(y_test, y_pred_test, average='weighted')

        # Calculate train accuracy and F1 score
        y_pred_train = predictions_train['prediction_label']
        train_accuracy = accuracy_score(y_train, y_pred_train)
        train_f1 = f1_score(y_train, y_pred_train, average='weighted')

        # Append the metrics for each model to the results DataFrame
        new_row = pd.DataFrame({
            'model': model_name,
            'train_accuracy': train_accuracy,
            'test_accuracy': test_accuracy,
            'train_f1': train_f1,
            'test_f1': test_f1,
            'TT': testing_time
        }, index=[0])

        goalpoacher_df = pd.concat([goalpoacher_df, new_row], ignore_index=True)


,Description,Value
0,Session id,42
1,Target,Class_Goal Poacher
2,Target type,Binary
3,Original data shape,"(91, 15)"
4,Transformed data shape,"(91, 15)"
5,Transformed train set shape,"(63, 15)"
6,Transformed test set shape,"(28, 15)"
7,Numeric features,14
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.8905,0.9346,0.7833,0.8333,0.7767,0.7285,0.7531,0.0310
et,Extra Trees Classifier,0.8905,0.9383,0.7333,0.8667,0.7733,0.7218,0.7446,0.1070
lr,Logistic Regression,0.8571,0.9358,0.7667,0.8167,0.7767,0.6734,0.6870,0.0380
ridge,Ridge Classifier,0.8571,0.9167,0.7167,0.7833,0.7300,0.6493,0.6652,0.0160
lda,Linear Discriminant Analysis,0.8571,0.9167,0.7167,0.7833,0.7300,0.6493,0.6652,0.0190
qda,Quadratic Discriminant Analysis,0.8286,0.8333,0.5833,0.7500,0.6371,0.5587,0.5807,0.0220
rf,Random Forest Classifier,0.8071,0.9058,0.6167,0.7583,0.6557,0.5397,0.5641,0.0860
svm,SVM - Linear Kernel,0.7952,0.8742,0.6500,0.6817,0.6229,0.5300,0.5684,0.0170
ada,Ada Boost Classifier,0.7762,0.8667,0.7167,0.6083,0.6424,0.4949,0.5195,0.0670
xgboost,Extreme Gradient Boosting,0.7500,0.8550,0.6333,0.6250,0.6090,0.4293,0.4466,0.0340


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.8905,0.9346,0.7833,0.8333,0.7767,0.7285,0.7531,0.0290


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,0.6833,0.6658,0.6167,0.5833,0.5657,0.3317,0.3524,0.0130


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.6524,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0120


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

In [32]:
goalpoacher_df

,model,train_accuracy,test_accuracy,train_f1,test_f1,TT
0,K Neighbors Classifier,0.923077,0.869565,0.922790,0.864803,0.031
1,Extra Trees Classifier,1.000000,0.913043,1.000000,0.913043,0.107
2,Logistic Regression,1.000000,0.956522,1.000000,0.956687,0.038
3,Ridge Classifier,0.890110,0.869565,0.890110,0.870062,0.016
4,Linear Discriminant Analysis,0.890110,0.869565,0.890110,0.870062,0.019
5,Random Forest Classifier,1.000000,0.826087,1.000000,0.822636,0.086
6,SVM - Linear Kernel,0.912088,0.956522,0.910599,0.956183,0.017
7,Ada Boost Classifier,1.000000,0.826087,1.000000,0.826087,0.067
8,Extreme Gradient Boosting,1.000000,0.913043,1.000000,0.913043,0.034
9,Gradient Boosting Classifier,1.000000,0.913043,1.000000,0.913043,0.070


## Target Man

In [33]:
import pandas as pd
from pycaret.classification import setup, compare_models, predict_model, finalize_model, pull
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import joblib

# Load the CSV file into a DataFrame
df = pd.read_csv('../data/players_df_sin_reco.csv')

# Select features (columns 7 to 20, 0-based index: 6 to 19)
X = df.iloc[:, 6:20]

# Select the last column from the end as the target
y = df.iloc[:, -1]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an empty DataFrame to store results
targetman_df = pd.DataFrame(columns=['model', 'train_accuracy', 'test_accuracy', 'train_f1', 'test_f1', 'TT'])

# Concatenate X_train and y_train into a DataFrame for PyCaret setup
df_target = pd.concat([X_train, y_train], axis=1)

# Setup PyCaret environment
clf = setup(data=df_target, target=y.name, session_id=42)

# Compare different models and store the results
best_model = compare_models()  # This returns the top model
compare_results = pull()  # Pulls all model results as a DataFrame

# Define the mapping from model names to PyCaret identifiers
model_mapping = {
    'K Neighbors Classifier': 'knn',
    'Random Forest Classifier': 'rf',
    'Logistic Regression': 'lr',
    'Decision Tree Classifier': 'dt',
    'Ada Boost Classifier': 'ada',
    'Gradient Boosting Classifier': 'gbc',
    'Naive Bayes': 'nb',
    'SVM - Linear Kernel': 'svm',
    'Extra Trees Classifier': 'et',
    'Light Gradient Boosting Machine': 'lightgbm',
    'Ridge Classifier': 'ridge',
    'Linear Discriminant Analysis': 'lda',
    'Extreme Gradient Boosting': 'xgboost',
    'Dummy Classifier': 'dummy'
}

# Loop over each model in the compare_results DataFrame
for idx, row in compare_results.iterrows():
    model_name = row['Model']
    testing_time = row['TT (Sec)']

    # Get the PyCaret identifier for the model
    model_identifier = model_mapping.get(model_name)

    # Instead of finalizing by the model name, select the model by identifier
    if model_identifier:
        best_model = compare_models(include=[model_identifier], n_select=1)

        # Finalize the model (train it on the entire dataset)
        final_model = finalize_model(best_model)

        # Predict on train and test sets
        predictions_train = predict_model(final_model, data=X_train)
        predictions_test = predict_model(final_model, data=X_test)

        # Calculate test accuracy and F1 score
        y_pred_test = predictions_test['prediction_label']
        test_accuracy = accuracy_score(y_test, y_pred_test)
        test_f1 = f1_score(y_test, y_pred_test, average='weighted')

        # Calculate train accuracy and F1 score
        y_pred_train = predictions_train['prediction_label']
        train_accuracy = accuracy_score(y_train, y_pred_train)
        train_f1 = f1_score(y_train, y_pred_train, average='weighted')

        # Append the metrics for each model to the results DataFrame
        new_row = pd.DataFrame({
            'model': model_name,
            'train_accuracy': train_accuracy,
            'test_accuracy': test_accuracy,
            'train_f1': train_f1,
            'test_f1': test_f1,
            'TT': testing_time
        }, index=[0])

        targetman_df = pd.concat([targetman_df, new_row], ignore_index=True)


,Description,Value
0,Session id,42
1,Target,Class_Target Man
2,Target type,Binary
3,Original data shape,"(91, 15)"
4,Transformed data shape,"(91, 15)"
5,Transformed train set shape,"(63, 15)"
6,Transformed test set shape,"(28, 15)"
7,Numeric features,14
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9690,1.0000,1.0000,0.9550,0.9746,0.9362,0.9437,0.0800
xgboost,Extreme Gradient Boosting,0.9690,0.9889,1.0000,0.9550,0.9746,0.9362,0.9437,0.0310
et,Extra Trees Classifier,0.9548,1.0000,1.0000,0.9350,0.9635,0.9058,0.9168,0.0700
knn,K Neighbors Classifier,0.9381,0.9681,1.0000,0.9100,0.9492,0.8725,0.8875,0.0210
ada,Ada Boost Classifier,0.9214,0.9889,0.9500,0.9350,0.9349,0.8320,0.8527,0.0520
lr,Logistic Regression,0.9190,0.9556,0.9750,0.9150,0.9353,0.8362,0.8592,0.0220
gbc,Gradient Boosting Classifier,0.9048,0.9167,0.9500,0.9100,0.9206,0.8058,0.8289,0.0490
lightgbm,Light Gradient Boosting Machine,0.8929,0.9722,0.9167,0.9017,0.9052,0.7808,0.7918,0.0370
ridge,Ridge Classifier,0.8857,0.9556,0.9083,0.8967,0.8984,0.7625,0.7716,0.0150
lda,Linear Discriminant Analysis,0.8857,0.9556,0.9083,0.8967,0.8984,0.7625,0.7716,0.0150


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9690,1.0000,1.0000,0.9550,0.9746,0.9362,0.9437,0.0880


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
svm,SVM - Linear Kernel,0.8690,0.9667,0.8583,0.9350,0.8659,0.7334,0.7724,0.0200


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.5881,0.5000,1.0000,0.5881,0.7382,0.0000,0.0000,0.0160


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

In [34]:
targetman_df

,model,train_accuracy,test_accuracy,train_f1,test_f1,TT
0,Random Forest Classifier,1.000000,0.739130,1.000000,0.725064,0.080
1,Extreme Gradient Boosting,1.000000,0.869565,1.000000,0.857369,0.031
2,Extra Trees Classifier,1.000000,0.782609,1.000000,0.762281,0.070
3,K Neighbors Classifier,0.956044,0.869565,0.955833,0.857369,0.021
4,Ada Boost Classifier,1.000000,0.826087,1.000000,0.816709,0.052
5,Logistic Regression,1.000000,0.869565,1.000000,0.866525,0.022
6,Gradient Boosting Classifier,1.000000,0.826087,1.000000,0.826087,0.049
7,Light Gradient Boosting Machine,1.000000,0.782609,1.000000,0.762281,0.037
8,Ridge Classifier,0.934066,0.739130,0.934066,0.701449,0.015
9,Linear Discriminant Analysis,0.934066,0.739130,0.934066,0.701449,0.015
